Notes:
- new data from remm se output
- old data from wfrc data portal
- before running I standarized the column names

Create a Feature Layer for each combination of Map Display Geography (4) x Variables (7) wherein the attribute fields are 

  Like this for Households (for the other 6 variables substitute hh_ with: pop_, ijb_, ojb, rjb_, tjb_,  hji_)

TAZID, city, county, med_dist, small_dist, dev_acres, hh_19, hh_20, hh_21…hh_50…old_hh_20, old_hh_25, old_hh_50
Sml_DistID, county, dev_acres, hh_19, hh_20, hh_21…hh_50…old_hh_20, old_hh_25, old_hh_50
Med_DistID, county, dev_acres, hh_19, hh_20, hh_21…hh_50…old_hh_20, old_hh_25, old_hh_50
CityID, county, dev_acres, hh_19, hh_20, hh_21…hh_50…old_hh_20, old_hh_25, old_hh_50


### For the map:
<!-- 1. obtain taz 832 and 900 shapefile -->
<!-- 2. attributes we need:
  - total households
  - total population
  - total jobs
  - industrial jobs(5,10) 
  - retail jobs(1,9) 
  - office jobs(6,4,3)
  - typical jobs (sum of three types, calculated)
  - HJ Intensity (this is (HH * 1.8) + Jobs) -->
<!-- 3. get taz outout from 2019 REMM -->
<!-- 4. get Official TAZ output from 2015 REMM (confirm TAZ version) -->
<!-- 5. Generate folder structure -->
6. Apportion new taz data to:
  - taz 9.0.0
  - medium district
  - city
  - small district (later)
7. Apportion old taz data to:
  - taz 9.0.0
  - medium district
  - city
  - small district (later)
8. create shapefiles/feature classes for each attribute (see #2)

### For the chart:
1. create a json for each record using the following geographies:
  - taz
  - medium district
  - city
  - small district (later)



In [2]:
import arcpy
from arcpy import env
import os
import glob
from arcgis import GIS
from arcgis.features import GeoAccessor
import pandas as pd

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

# show all columns
pd.options.display.max_columns = None

# pd.DataFrame.spatial.from_featureclass(???)
# df.spatial.to_featureclass(location=???,sanitize_columns=False)

In [3]:
# import datasets
# parcels = r'.\\Inputs\\remm_base_year_20220513.gdb\\parcels_2019'
taz832 = r".\Inputs\TAZ_832_ID_Only.shp"
taz900 = r".\Inputs\TAZ_900.shp"
cities = r".\Inputs\CityArea.shp"
# small_districts = ???
# regions_df = pd.DataFrame.spatial.from_featureclass(regions_shp)

taz832_sdf = pd.DataFrame.spatial.from_featureclass(taz832)
taz900_sdf = pd.DataFrame.spatial.from_featureclass(taz900)

In [4]:
pm_folder = r".\New_SE_Data"
new_taz_se = glob.glob(os.path.join(pm_folder,'SE_WF_*.csv'))
len(new_taz_se)

32

In [40]:
old_se_folder = r".\Old_SE_Data"
old_taz_se = glob.glob(os.path.join(old_se_folder,'*_TAZ.csv'))
old_city_se = glob.glob(os.path.join(old_se_folder,'*_City_Area.csv'))

In [6]:
outputs = r'.\Outputs'
if not os.path.exists(outputs):
    os.makedirs(outputs)

scratch = os.path.join(outputs, 'scratch.gdb')
if not arcpy.Exists(scratch):
    arcpy.CreateFileGDB_management(outputs, 'scratch.gdb')

map_folder = os.path.join(outputs, "map")
if not os.path.exists(map_folder):
    os.makedirs(map_folder)

chart_folder = os.path.join(outputs, "chart")
if not os.path.exists(chart_folder):
    os.makedirs(chart_folder)

In [8]:
# year = os.path.splitext(os.path.basename(new_taz_se[0]))[0].split('_')[-1]
# x = pd.read_csv(new_taz_se[0])
# x['OFFI'] = x['OFFI'] + x['GVED'] + x['HLTH']
# x['TPCL'] = x['OFFI'] + x['RETEMP'] + x['INDEMP']
# x['HJI'] = x['TOTHH']*1.8 + x['TOTEMP']
# x = x[[';TAZID', 'TOTHH', 'HHPOP', 'TOTEMP','RETEMP', 'INDEMP', 'OFFI', 'TPCL', 'HJI']].copy()
# x.columns = ['TAZID', f'HH_{year}', f'POP_{year}', f'EMP_{year}', f'RTL_{year}', f'IND_{year}', f'OFFI_{year}', f'TPCL_{year}', f'HJI_{year}']
# x


### Process New SE Data

In [109]:
base = taz832_sdf[['TAZID','SHAPE']].copy()
base.shape

for csv in new_taz_se:

    year = os.path.splitext(os.path.basename(csv))[0].split('_')[-1]
    df = pd.read_csv(csv)
    df['OFFI'] = df['OFFI'] + df['GVED'] + df['HLTH']
    df['TPCL'] = df['OFFI'] + df['RETEMP'] + df['INDEMP']
    df['HJI'] = df['TOTHH']*1.8 + df['TOTEMP']
    df = df[[';TAZID', 'TOTHH', 'HHPOP', 'TOTEMP','RETEMP', 'INDEMP', 'OFFI', 'TPCL', 'HJI']].copy()
    df.columns = ['TAZID', f'HH_{year}', f'POP_{year}', f'EMP_{year}', f'RTL_{year}', f'IND_{year}', f'OFFI_{year}', f'TPCL_{year}', f'HJI_{year}']
    
    base = base.merge(df, on='TAZID', how='left')

new_se = base
new_se.rename({'TAZID': 'TAZID832'}, axis=1, inplace=True)

In [121]:
new_se.shape

(2881, 258)

In [112]:
# export
new_se_taz_832 = os.path.join(scratch, '_01_new_se_taz_832')
new_se.spatial.to_featureclass(location=new_se_taz_832,sanitize_columns=False)

'e:\\Projects\\REMM-Process-Progression-Metrics-For-Web\\Outputs\\scratch.gdb\\_01_new_se_taz_832'

### Process Old SE Data

In [42]:
se_dict =  {'TOTHH':'HH',    
            'HHPOP':'POP',
            'RETEMP':'RTL',
            'TOTEMP':'TPCL',
            'ALLEMP':'EMP',
            'INDEMP':'IND',
            'OTHEMP':'OFFI'}

In [93]:
# test = pd.read_csv(old_taz_se[0])
# category = se_dict[test.iloc[0]['SECategory']]
# val_cols = [col for col in list(test.columns) if 'YEAR' in col and  'D' not in col and int(col[4:]) >= 2019]
# test = test[['CO_TAZID'] + val_cols]
# new_val_cols = [col.replace('YEAR',category + '_') for col in val_cols]
# test.columns = ['CO_TAZID'] + new_val_cols
# test

In [114]:
old_base = taz832_sdf[['CO_TAZID','SHAPE']].copy()
old_base.shape

for csv in old_taz_se:

    df = pd.read_csv(csv)
    category = se_dict[df.iloc[0]['SECategory']]
    val_cols = [col for col in list(df.columns) if 'YEAR' in col and  'D' not in col and int(col[4:]) >= 2019]
    df = df[['CO_TAZID'] + val_cols]
    new_val_cols = [col.replace('YEAR',category + '_') for col in val_cols]
    df.columns = ['CO_TAZID'] + new_val_cols
    old_base = old_base.merge(df, on='CO_TAZID',how='left')

old_se = old_base
old_se.rename({'CO_TAZID': 'COTAZID832'}, axis=1, inplace=True)

In [115]:
# calculate household jobs index
for year in range(2019,2051):
    old_se['HJI_{}'.format(year)] = (old_se['HH_{}'.format(year)] * 1.8) + old_se['EMP_{}'.format(year)]

In [116]:
old_se.shape

(2881, 258)

In [117]:
# export
old_se_taz_832 = os.path.join(scratch, '_02_old_se_taz_832')
old_se.spatial.to_featureclass(location=old_se_taz_832,sanitize_columns=False)

'e:\\Projects\\REMM-Process-Progression-Metrics-For-Web\\Outputs\\scratch.gdb\\_02_old_se_taz_832'

## Identity

In [118]:
# compare output dimensions
old_se.shape == new_se.shape

True

In [138]:
# merge geometry between taz 832 and output geometries (taz900, )
taz900_identity = arcpy.Identity_analysis(taz900, new_se_taz_832, os.path.join(scratch, '_03_taz832_taz900_identity'))
arcpy.management.DeleteField(taz900_identity,['TAZID', 'DISTMED'], "KEEP_FIELDS")

cities_identity = arcpy.Identity_analysis(cities, new_se_taz_832,os.path.join(scratch, '_03_taz832_cities_identity'))
arcpy.management.DeleteField(cities_identity,'CityArea', "KEEP_FIELDS")

<Result '.\\Outputs\\scratch.gdb\\_03_taz832_cities_identity'>

## Apportion the attributes from old geometry to the new ones

- The Apportion tool in arcpy is still lame 
- we have to run a command (apportion_command_for_arcgis_pro.txt) in the arcgis pro python window to run it

In [139]:
# apportion = arcpy.ApportionPolygon_analysis(new_se_taz_832, app_columns, identity, os.path.join(outputs, 'new_se_apportion_to_taz900.shp'), "AREA", "", "", "MAINTAIN_GEOMETRIES")
new_taz_apportion = os.path.join(scratch,'_05_new_se_apportion_to_taz900')
new_city_apportion = os.path.join(scratch,'_05_new_se_apportion_to_cities')
old_taz_apportion = os.path.join(scratch,'_05_old_se_apportion_to_taz900')
old_city_apportion = os.path.join(scratch,'_05_old_se_apportion_to_cities')

## Dissolve to get to desired geometry

In [140]:
app_columns = list(new_se.columns)[2:]
stat_fields = [[col,'SUM'] for col in app_columns]

#########
# New
#########

# taz (9.0.0)
new_se_taz900_dissolve = arcpy.Dissolve_management(new_taz_apportion, os.path.join(scratch, '_06_new_se_taz900_dissolve'),
                          'TAZID', stat_fields, "MULTI_PART")

# distmed (9.0.0)
new_se_distmed_dissolve = arcpy.Dissolve_management(new_taz_apportion, os.path.join(scratch, '_06_new_se_distmed_dissolve'),
                          'DISTMED', stat_fields, "MULTI_PART")

# city area
new_se_cityarea_dissolve = arcpy.Dissolve_management(new_city_apportion, os.path.join(scratch, '_06_new_se_cityarea_dissolve'),
                          'CITYAREA', stat_fields, "MULTI_PART")




In [141]:
#########
# Old
#########

# taz (9.0.0)
old_se_taz900_dissolve = arcpy.Dissolve_management(old_taz_apportion, os.path.join(scratch, '_06_old_se_taz900_dissolve'),
                          'TAZID', stat_fields, "MULTI_PART")

# distmed (9.0.0)
old_se_distmed_dissolve = arcpy.Dissolve_management(old_taz_apportion, os.path.join(scratch, '_06_old_se_distmed_dissolve'),
                          'DISTMED', stat_fields, "MULTI_PART")

# city area
old_se_cityarea_dissolve = arcpy.Dissolve_management(old_city_apportion, os.path.join(scratch, '_06_old_se_cityarea_dissolve'),
                          'CITYAREA', stat_fields, "MULTI_PART")


In [142]:
new_se_taz900_dissolve_df = pd.DataFrame.spatial.from_featureclass(new_se_taz900_dissolve[0])
new_se_distmed_dissolve_df = pd.DataFrame.spatial.from_featureclass(new_se_distmed_dissolve[0])
new_se_cityarea_dissolve_df = pd.DataFrame.spatial.from_featureclass(new_se_cityarea_dissolve[0])
old_se_taz900_dissolve_df = pd.DataFrame.spatial.from_featureclass(old_se_taz900_dissolve[0])
old_se_distmed_dissolve_df = pd.DataFrame.spatial.from_featureclass(old_se_distmed_dissolve[0])
old_se_cityarea_dissolve_df = pd.DataFrame.spatial.from_featureclass(old_se_cityarea_dissolve[0])

In [146]:
app_columns
to_replace = ['SUM_'+ col for col in app_columns]
replace_dict = dict(zip(to_replace, app_columns))

{'SUM_HH_2019': 'HH_2019',
 'SUM_POP_2019': 'POP_2019',
 'SUM_EMP_2019': 'EMP_2019',
 'SUM_RTL_2019': 'RTL_2019',
 'SUM_IND_2019': 'IND_2019',
 'SUM_OFFI_2019': 'OFFI_2019',
 'SUM_TPCL_2019': 'TPCL_2019',
 'SUM_HJI_2019': 'HJI_2019',
 'SUM_HH_2020': 'HH_2020',
 'SUM_POP_2020': 'POP_2020',
 'SUM_EMP_2020': 'EMP_2020',
 'SUM_RTL_2020': 'RTL_2020',
 'SUM_IND_2020': 'IND_2020',
 'SUM_OFFI_2020': 'OFFI_2020',
 'SUM_TPCL_2020': 'TPCL_2020',
 'SUM_HJI_2020': 'HJI_2020',
 'SUM_HH_2021': 'HH_2021',
 'SUM_POP_2021': 'POP_2021',
 'SUM_EMP_2021': 'EMP_2021',
 'SUM_RTL_2021': 'RTL_2021',
 'SUM_IND_2021': 'IND_2021',
 'SUM_OFFI_2021': 'OFFI_2021',
 'SUM_TPCL_2021': 'TPCL_2021',
 'SUM_HJI_2021': 'HJI_2021',
 'SUM_HH_2022': 'HH_2022',
 'SUM_POP_2022': 'POP_2022',
 'SUM_EMP_2022': 'EMP_2022',
 'SUM_RTL_2022': 'RTL_2022',
 'SUM_IND_2022': 'IND_2022',
 'SUM_OFFI_2022': 'OFFI_2022',
 'SUM_TPCL_2022': 'TPCL_2022',
 'SUM_HJI_2022': 'HJI_2022',
 'SUM_HH_2023': 'HH_2023',
 'SUM_POP_2023': 'POP_2023',
 'SUM_EM

In [ ]:
# store each df in list
# get name/geography
# pair old and new tables
# rename old columns
# create separate dataframe for each category, remove category from field names
# export each category to separate shapefile to map folder
# process to json to chart folder